In [1]:
!pip3 install gensim

In [74]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import pandas as pd
import gensim
import pprint
from gensim import corpora
from gensim.utils import simple_preprocess
import gensim.models as models
import numpy as np
from itertools import chain
import matplotlib.pyplot as plt
from decimal import Decimal
!pwd

/u/arunas/6.864/code


In [75]:
train = pd.read_csv('../data/train_split_v1.csv')
test = pd.read_csv('../data/test_split_v1.csv')
val = pd.read_csv('../data/val_split_v1.csv')

tweets = pd.read_csv('../data/tweet_text2021-11-04.csv')
notes = pd.read_csv('../data/notes-00000.tsv', sep='\t')
ratings = pd.read_csv('../data/ratings-00000.tsv', sep='\t')


In [76]:
cols = ['tweetId', 'title']
len(train.index), len(test.index), len(val.index)

(12688, 1587, 1586)

In [77]:

train = train[['noteId', 'tweetId', 'summary']].join(tweets[['id', 'text']].set_index('id'), on='tweetId', how='left', lsuffix='_x', rsuffix='_y')
test = test[['noteId', 'tweetId', 'summary']].join(tweets[['id', 'text']].set_index('id'), on='tweetId', how='left', lsuffix='_x', rsuffix='_y')
val = val[['noteId', 'tweetId', 'summary']].join(tweets[['id', 'text']].set_index('id'), on='tweetId', how='left', lsuffix='_x', rsuffix='_y')

In [78]:
print(len(train.index), len(test.index), len(val.index))
print(train.columns)

# train.columns, test.columns, val.columns
# len(train.index), len(test.index), len(val.index)
# train.columns

12688 1587 1586
Index(['noteId', 'tweetId', 'summary', 'text'], dtype='object')


# Labels

In [79]:
ratings.columns

Index(['noteId', 'participantId', 'createdAtMillis', 'version', 'agree',
       'disagree', 'helpful', 'notHelpful', 'helpfulnessLevel', 'helpfulOther',
       'helpfulInformative', 'helpfulClear', 'helpfulEmpathetic',
       'helpfulGoodSources', 'helpfulUniqueContext', 'helpfulAddressesClaim',
       'helpfulImportantContext', 'notHelpfulOther', 'notHelpfulIncorrect',
       'notHelpfulSourcesMissingOrUnreliable',
       'notHelpfulOpinionSpeculationOrBias', 'notHelpfulMissingKeyPoints',
       'notHelpfulOutdated', 'notHelpfulHardToUnderstand',
       'notHelpfulArgumentativeOrInflammatory', 'notHelpfulOffTopic',
       'notHelpfulSpamHarassmentOrAbuse', 'notHelpfulIrrelevantSources'],
      dtype='object')

In [80]:
ratings_subset = ratings[['noteId', 'helpfulnessLevel']].dropna()

helpful_dict = {'HELPFUL' : 1, 'NOT_HELPFUL': 0, 'SOMEWHAT_HELPFUL': 0.5}
ratings_subset['label'] = ratings_subset['helpfulnessLevel'].apply(lambda x: helpful_dict[x])

ratings_subset = ratings_subset.groupby('noteId')['label'].mean().reset_index()

In [81]:
train = train.join(ratings_subset.set_index('noteId'), on='noteId', how='inner', lsuffix='_x', rsuffix='_y')
test = test.join(ratings_subset.set_index('noteId'), on='noteId', how='inner', lsuffix='_x', rsuffix='_y')
val = val.join(ratings_subset.set_index('noteId'), on='noteId', how='inner', lsuffix='_x', rsuffix='_y')

print(len(train.index), len(test.index), len(val.index))
print(train.columns)



5296 626 681
Index(['noteId', 'tweetId', 'summary', 'text', 'label'], dtype='object')


In [82]:
train['tweet_note']=[(f"<SEP>{tweet}<SEP_BW>{note}</SEP>", label) for tweet, note, label in zip(train['text'], train['summary'], train['label'])]
test['tweet_note']=[(f"<SEP>{tweet}<SEP_BW>{note}</SEP>", label) for tweet, note, label in zip(test['text'], test['summary'], test['label'])]
val['tweet_note']=[(f"<SEP>{tweet}<SEP_BW>{note}</SEP>", label) for tweet, note, label in zip(val['text'], val['summary'], val['label'])]

In [83]:
documents = [doc[0] for doc in train['tweet_note']]
label = [int(doc[1] > 0) for doc in train['tweet_note']]
# label = label/np.linalg.norm(label)
documents_tokenized = [simple_preprocess(doc) for doc in documents]
dictionary = corpora.Dictionary()
BoW_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in documents_tokenized]
tfidf = models.TfidfModel(BoW_corpus, smartirs='ntc')

2021-11-14 23:42:11,676 : INFO : collecting document frequencies
2021-11-14 23:42:11,678 : INFO : PROGRESS: processing document #0
2021-11-14 23:42:11,787 : INFO : TfidfModel lifecycle event {'msg': 'calculated IDF weights for 5296 documents and 21298 features (286258 matrix non-zeros)', 'datetime': '2021-11-14T23:42:11.787283', 'gensim': '4.1.2', 'python': '3.8.10 (default, Sep 28 2021, 16:10:42) \n[GCC 9.3.0]', 'platform': 'Linux-5.8.0-50-generic-x86_64-with-glibc2.29', 'event': 'initialize'}


In [54]:
BoW_list = list(tfidf[BoW_corpus])
print(documents[0], BoW_list[0])

<SEP>CDC is arm of DNC.<SEP_BW>This is a lie. CDC's parent agency is the United States Department of Health and Human Services https://www.cdc.gov/about/organization/cio.htm</SEP> [(0, 0.08988692622437675), (1, 0.24566794428290145), (2, 0.020446296995960414), (3, 0.28351854836142576), (4, 0.4003297759648584), (5, 0.37182060431521463), (6, 0.2040974617016572), (7, 0.32859469269146535), (8, 0.09615205747374914), (9, 0.11769958344217324), (10, 0.23291702855313706), (11, 0.005631047851479717), (12, 0.18727136156904245), (13, 0.053834621105222795), (14, 0.1611418940090403), (15, 0.040645640829613465), (16, 0.22708555416381293), (17, 0.33912146448232344), (18, 1.7813607861185632e-05), (19, 8.906803930592816e-06), (20, 0.21633987299230467), (21, 0.12632217753872652), (22, 0.005882594181078636), (23, 0.04249244820027498), (24, 0.1659919202521891), (25, 0.030481089841312876)]


# Determining the stop word list

In [55]:
sentence_lengths = [len(doc) for doc in tfidf[BoW_corpus]]
sent_len = np.percentile(sentence_lengths, 95)
print(sent_len)
stop_list = set()

81.0


In [56]:
for doc in tfidf[BoW_corpus]:
    doc.sort(key=lambda x: x[1])
    a = [dictionary[word[0]] for word in doc[:5]]
    stop_list.update(a)

print(stop_list)

{'in', 'with', 'they', 'are', 'evidence', 'out', 'trump', 'to', 'that', 'sense', 're', 'we', 'don', 'www', 'on', 'or', 'get', 'yes', 'back', 'you', 'your', 'longer', 'from', 'about', 'at', 'no', 'https', 'of', 'if', 'men', 'it', 'fact', 'than', 'sep_bw', 'tweet', 'people', 'joke', 'and', 'like', 'west', 'for', 'self', 'theft', 'because', 'real', 'covid', 'clearly', 'off', 'an', 'by', 'org', 'did', 'co', 'sep', 'the', 'is', 'com', 'not', 'he', 'have', 'would', 'has', 'this', 'weeks', 'biden', 'was', 'gov', 'as', 'coronavirus', 'who', 'be', 'cdc'}


# Doc2Vec -- get embeddings



In [57]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=documents_tokenized, vector_size=int(sent_len), window=10, min_count=1, workers=4)
documents[0], documents_tokenized[0]

model.save("word2vec.model")

2021-11-12 19:58:33,147 : INFO : collecting all words and their counts
2021-11-12 19:58:33,148 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-11-12 19:58:33,238 : INFO : collected 21298 word types from a corpus of 369656 raw words and 5296 sentences
2021-11-12 19:58:33,240 : INFO : Creating a fresh vocabulary
2021-11-12 19:58:33,332 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 21298 unique words (100.0%% of original 21298, drops 0)', 'datetime': '2021-11-12T19:58:33.332330', 'gensim': '4.1.2', 'python': '3.8.10 (default, Sep 28 2021, 16:10:42) \n[GCC 9.3.0]', 'platform': 'Linux-5.8.0-50-generic-x86_64-with-glibc2.29', 'event': 'prepare_vocab'}
2021-11-12 19:58:33,333 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 369656 word corpus (100.0%% of original 369656, drops 0)', 'datetime': '2021-11-12T19:58:33.333364', 'gensim': '4.1.2', 'python': '3.8.10 (default, Sep 28 2021, 16:10:42) \n[GCC 9.3.0]', 'pla

In [58]:
vector = model.wv['biden']
sims = model.wv.most_similar('biden', topn=10)
sims

[('president', 0.9502883553504944),
 ('joe', 0.9487913250923157),
 ('trump', 0.941845715045929),
 ('delaware', 0.9114742279052734),
 ('provision', 0.9030532836914062),
 ('electors', 0.9025029540061951),
 ('kamala', 0.8874210119247437),
 ('donald', 0.8859433531761169),
 ('hunter', 0.8781907558441162),
 ('pompeo', 0.8768683671951294)]

In [59]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(documents_tokenized)]
model = Doc2Vec(documents, vector_size=int(sent_len), window=10, min_count=1, workers=4)

# common_texts, documents

2021-11-12 19:58:40,216 : INFO : collecting all words and their counts
2021-11-12 19:58:40,217 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-11-12 19:58:40,303 : INFO : collected 21298 word types and 5296 unique tags from a corpus of 5296 examples and 369656 words
2021-11-12 19:58:40,304 : INFO : Creating a fresh vocabulary
2021-11-12 19:58:40,395 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=1 retains 21298 unique words (100.0%% of original 21298, drops 0)', 'datetime': '2021-11-12T19:58:40.395791', 'gensim': '4.1.2', 'python': '3.8.10 (default, Sep 28 2021, 16:10:42) \n[GCC 9.3.0]', 'platform': 'Linux-5.8.0-50-generic-x86_64-with-glibc2.29', 'event': 'prepare_vocab'}
2021-11-12 19:58:40,396 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 369656 word corpus (100.0%% of original 369656, drops 0)', 'datetime': '2021-11-12T19:58:40.396471', 'gensim': '4.1.2', 'python': '3.8.10 (default, Sep 28 2021, 16:10:42) \

In [60]:
from gensim.test.utils import get_tmpfile
fname = get_tmpfile("doc2vec_tweets_notes")
model.save(fname)

model = Doc2Vec.load(fname)

2021-11-12 19:58:48,952 : INFO : Doc2Vec lifecycle event {'fname_or_handle': '/tmp/tmpwu_bnr08/doc2vec_tweets_notes', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2021-11-12T19:58:48.952191', 'gensim': '4.1.2', 'python': '3.8.10 (default, Sep 28 2021, 16:10:42) \n[GCC 9.3.0]', 'platform': 'Linux-5.8.0-50-generic-x86_64-with-glibc2.29', 'event': 'saving'}
2021-11-12 19:58:48,952 : INFO : not storing attribute cum_table
2021-11-12 19:58:48,982 : INFO : saved /tmp/tmpwu_bnr08/doc2vec_tweets_notes
2021-11-12 19:58:48,983 : INFO : loading Doc2Vec object from /tmp/tmpwu_bnr08/doc2vec_tweets_notes
2021-11-12 19:58:48,996 : INFO : loading dv recursively from /tmp/tmpwu_bnr08/doc2vec_tweets_notes.dv.* with mmap=None
2021-11-12 19:58:48,997 : INFO : loading wv recursively from /tmp/tmpwu_bnr08/doc2vec_tweets_notes.wv.* with mmap=None
2021-11-12 19:58:48,998 : INFO : setting ignored attribute cum_table to None
2021-11-12 19:58:49,359 : INFO : Doc2Vec lifecycle 

# Training

In [61]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as torch_data
from tqdm import tqdm
import matplotlib.pyplot as plt

In [62]:
# for i in range(len(label)):
#     print(label[i])
data = [(torch.from_numpy((model.infer_vector(documents_tokenized[i]))).float().unsqueeze(1), int(label[i])) for i in range(len(documents_tokenized))]

print(type(data[0][0]), type(data[0][1]))
# print(data[0], data[0].shape)
# print(data[1], data[1].shape)
# print(data[0][0].shape)

<class 'torch.Tensor'> <class 'int'>


In [63]:
device = torch.device('cuda:2')  # run on colab gpu

# class NeuralNetwork(nn.Module):
#     def __init__(self):
#         super(NeuralNetwork, self).__init__()
#         self.flatten = nn.Flatten()
#         self.linear_relu_stack = nn.Sequential(
#             nn.Linear(81, 40),
#             nn.ReLU(),
#             nn.Linear(40, 40),
#             nn.ReLU(),
#             nn.Linear(40, 1),
#         )

#     def forward(self, x):
#         x = x.to(device)
#         x = self.flatten(x)
#         logits = self.linear_relu_stack(x)
#         return logits

class NeuralNetwork(torch.nn.Module):
    def __init__(self, input_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        x = x.squeeze(0)
        outputs = torch.sigmoid(self.linear(x.T))
        return outputs

In [64]:
CUDA_LAUNCH_BLOCKING=1
n_batch = 100
nnModel = NeuralNetwork(data[0][0].shape[0], 2).float().to(device)
opt = optim.Adam(nnModel.parameters(), lr=0.001)

loader = torch_data.DataLoader(data, batch_size=100, shuffle=True)

loss_fn = nn.CrossEntropyLoss()
n_epochs = 500
losses = []  # Potentially useful for debugging (loss should go down!)
for epoch in tqdm(range(n_epochs)):
    epoch_loss = 0
    for x, y in loader:
        x = x.float().to(device)
        y = y.to(device)
        
        opt.zero_grad()
        preds = nnModel.forward(x.T).to(device)
        loss = loss_fn(preds, y)
        loss.backward()
        opt.step()

        epoch_loss += loss.item()
    losses.append(epoch_loss)

print(losses)
    
PATH = './word2vec.pth'
torch.save(nnModel.state_dict(), PATH)

100%|██████████| 500/500 [00:34<00:00, 14.47it/s]

[34.98855006694794, 32.501944959163666, 30.62615019083023, 29.24514675140381, 28.20510596036911, 27.40751326084137, 26.783091634511948, 26.27863672375679, 25.868837594985962, 25.524880290031433, 25.235499262809753, 24.984737932682037, 24.76930332183838, 24.581441909074783, 24.41574537754059, 24.268038481473923, 24.135708063840866, 24.017120629549026, 23.90847772359848, 23.809518069028854, 23.721173495054245, 23.639677226543427, 23.56456595659256, 23.496120363473892, 23.429496496915817, 23.370623648166656, 23.313936978578568, 23.259805411100388, 23.213137477636337, 23.16669636964798, 23.122585117816925, 23.082998543977737, 23.043576329946518, 23.00527858734131, 22.972083568572998, 22.93942102789879, 22.907775729894638, 22.879473447799683, 22.851094156503677, 22.82430651783943, 22.800704032182693, 22.774579912424088, 22.750482887029648, 22.72758713364601, 22.705737620592117, 22.6850728392601, 22.667626291513443, 22.64545527100563, 22.628178030252457, 22.610565096139908, 22.59292331337928

In [72]:
test_doc = [doc[0] for doc in test['tweet_note']]
test_label = [(doc[1] > 0) for doc in test['tweet_note']]

print(test_label)

test_tokens = [simple_preprocess(doc) for doc in test_doc]
test_BoW_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in test_tokens]
tfidf = models.TfidfModel(test_BoW_corpus, smartirs='ntc')

test_model = Word2Vec(sentences=test_tokens, vector_size=int(sent_len), window=10, min_count=1, workers=4)

test_model.save("word2vec_test.model")

test_documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(test_tokens)]
test_model = Doc2Vec(test_documents, vector_size=int(sent_len), window=10, min_count=1, workers=4)


fname = get_tmpfile("doc2vec_tweets_notes_test")
test_model.save(fname)

test_model = Doc2Vec.load(fname)

2021-11-12 20:05:38,298 : INFO : collecting document frequencies
2021-11-12 20:05:38,299 : INFO : PROGRESS: processing document #0
2021-11-12 20:05:38,318 : INFO : TfidfModel lifecycle event {'msg': 'calculated IDF weights for 626 documents and 22599 features (33395 matrix non-zeros)', 'datetime': '2021-11-12T20:05:38.318771', 'gensim': '4.1.2', 'python': '3.8.10 (default, Sep 28 2021, 16:10:42) \n[GCC 9.3.0]', 'platform': 'Linux-5.8.0-50-generic-x86_64-with-glibc2.29', 'event': 'initialize'}
2021-11-12 20:05:38,320 : INFO : collecting all words and their counts
2021-11-12 20:05:38,321 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-11-12 20:05:38,330 : INFO : collected 6437 word types from a corpus of 43135 raw words and 626 sentences
2021-11-12 20:05:38,330 : INFO : Creating a fresh vocabulary
2021-11-12 20:05:38,358 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 6437 unique words (100.0%% of original 6437, drops 0)', 'datetim

[True, False, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True

2021-11-12 20:05:38,506 : INFO : estimated required memory for 6437 words and 81 dimensions: 7389676 bytes
2021-11-12 20:05:38,506 : INFO : resetting layer weights
2021-11-12 20:05:38,513 : INFO : Word2Vec lifecycle event {'update': False, 'trim_rule': 'None', 'datetime': '2021-11-12T20:05:38.513871', 'gensim': '4.1.2', 'python': '3.8.10 (default, Sep 28 2021, 16:10:42) \n[GCC 9.3.0]', 'platform': 'Linux-5.8.0-50-generic-x86_64-with-glibc2.29', 'event': 'build_vocab'}
2021-11-12 20:05:38,514 : INFO : Word2Vec lifecycle event {'msg': 'training model with 4 workers on 6437 vocabulary and 81 features, using sg=0 hs=0 sample=0.001 negative=5 window=10 shrink_windows=True', 'datetime': '2021-11-12T20:05:38.514474', 'gensim': '4.1.2', 'python': '3.8.10 (default, Sep 28 2021, 16:10:42) \n[GCC 9.3.0]', 'platform': 'Linux-5.8.0-50-generic-x86_64-with-glibc2.29', 'event': 'train'}
2021-11-12 20:05:38,554 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-11-12 20:05:38,555 :

2021-11-12 20:05:39,391 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-11-12 20:05:39,394 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-11-12 20:05:39,395 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-12 20:05:39,404 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-12 20:05:39,404 : INFO : EPOCH - 4 : training on 43135 raw words (34701 effective words) took 0.1s, 375943 effective words/s
2021-11-12 20:05:39,487 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-11-12 20:05:39,491 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-11-12 20:05:39,492 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-12 20:05:39,501 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-12 20:05:39,501 : INFO : EPOCH - 5 : training on 43135 raw words (34789 effective words) took 0.1s, 371875 effective words/s
2021-1

In [73]:
test_data = [(torch.from_numpy((model.infer_vector(test_tokens[i]))).float().unsqueeze(1), int(test_label[i])) for i in range(len(test_tokens))]
# print([doc[1] for doc in test_data])

test_dataloader = torch_data.DataLoader(test_data, batch_size=100, shuffle=True)

# # testNnModel = torch.load(PATH)
# # testNnModel.eval()

nnModel.eval()

total_acc, total_count = 0, 0

with torch.no_grad():
    for test_x, test_y in test_dataloader:
        test_x = test_x.to(device)
        test_y = test_y.to(device)
        pred_test = nnModel(test_x.T)
#         pred_test = torch.broadcast_to(torch.tensor([[0.0,0.9]]), (test_y.shape[0], 2))
        print(pred_test, test_y, test_y.shape)
        loss = loss_fn(pred_test.to(device), test_y.to(device))
        total_acc += (pred_test.argmax(1).to(device) == test_y.to(device)).sum().item()
        total_count += test_y.size(0)
print(total_acc/total_count, loss)



tensor([[1.0561e-03, 9.9888e-01],
        [5.4742e-06, 9.9999e-01],
        [2.9068e-05, 9.9997e-01],
        [4.2577e-06, 1.0000e+00],
        [3.9784e-03, 9.9586e-01],
        [1.3268e-06, 1.0000e+00],
        [1.0793e-08, 1.0000e+00],
        [2.6668e-04, 9.9971e-01],
        [1.2678e-03, 9.9867e-01],
        [1.1388e-07, 1.0000e+00],
        [8.2403e-06, 9.9999e-01],
        [2.4479e-04, 9.9972e-01],
        [1.1644e-04, 9.9987e-01],
        [3.3163e-08, 1.0000e+00],
        [8.5275e-01, 1.4073e-01],
        [1.2696e-05, 9.9998e-01],
        [6.8761e-07, 1.0000e+00],
        [2.0029e-08, 1.0000e+00],
        [2.9269e-05, 9.9997e-01],
        [3.1770e-05, 9.9997e-01],
        [9.5481e-05, 9.9990e-01],
        [2.4732e-07, 1.0000e+00],
        [1.5096e-06, 1.0000e+00],
        [1.9371e-03, 9.9792e-01],
        [4.5934e-04, 9.9946e-01],
        [8.2450e-01, 1.7131e-01],
        [1.7885e-03, 9.9780e-01],
        [5.1111e-05, 9.9994e-01],
        [1.4477e-05, 9.9998e-01],
        [2.255